In [ ]:
import sys
sys.path.append("../tram")
sys.path.append("..")

# numerics imports
import numpy as np
from progressbar import progressbar

from scipy.sparse.linalg import eigsh

from transition_manifold import TransitionManifold
from sklearn.kernel_approximation import RBFSampler

class LinearRFFManifold(TransitionManifold):

    def __init__(self, system, xtest, t, dt, M,  epsi=1., rffdim=100):
        self.system = system
        self.xtest = xtest
        self.t = t
        self.dt = dt
        self.M = M
        self.epsi = epsi 
        self.sample = None
        self.rffdim = rffdim

    def computeRC(self):
        
        n_points = self.xtest.shape[0] # number of start points
        dimension = self.xtest.shape[1]

        # compute the time evolution of all test points at once, for performance reasons
        #x0 = np.tile(self.xtest, (self.M,1))
        x0 = np.repeat(self.xtest, self.M, axis=0)
        
        #pointclouds is of shape (self.M * n_points ) x dimension
        pointclouds = self.system.computeBurst(self.t, self.dt, x0, showprogress=True)
        #transform endpoints to Fourier domain        
        sampler = RBFSampler(n_components=self.rffdim)
        sampler.fit(pointclouds)
        self.sampler = sampler
        pointclouds = sampler.transform(pointclouds)
         
        #reshape to blocks where k*[1:M] intervals in rows correspond to start point k
        #idx = np.arange(self.M * n_points)
        #idx_new = idx.reshape(n_points, self.M).T.ravel()
        #pointclouds = pointclouds[idx_new, :]
        
        #compute RFF mean embeddings
        embedded = pointclouds.reshape(n_points, self.M, self.rffdim).sum(axis=1) / self.M
        
        #centering
        mean = embedded.sum(axis=0) / n_points
        embedded = embedded - mean

        cov = (1/n_points) * embedded.T @ embedded # rff dim x rff dim

        print("Compute linear RFF RC")
        _, vec = eigsh(cov, k=1, which="LM")
        return vec.real, sampler, embedded

In [ ]:
# TM imports
import gradient_system as gs
from demos.doublewell import potential

# setting up the system
domain = np.array([[-2, -1], [2, 1]])
beta = 1 # inverse energy
potential = potential.Potential()
system = gs.GradientSystem(potential, domain, beta)

# points in which to compute the reaction coordinate
nTestpoints = 32
xtest = system.generateTestpoints(nTestpoints, 'grid')

# points to visualize the potential
nPotpoints = 64
xpot = system.generateTestpoints(nPotpoints, 'grid')

TM=LinearRFFManifold(system,xtest, 1, 0.001, 100, .01, rffdim=1000)
vec, sampler, embedded = TM.computeRC()

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure()
ax = fig.add_subplot(111)

test_embedded = sampler.transform(xtest)

# visualize RC
ax.pcolormesh( xtest[:,0].reshape(nTestpoints, -1),
               xtest[:,1].reshape(nTestpoints, -1),
               (test_embedded @ vec).reshape(nTestpoints,-1),
                 shading='flat',
                 )

plt.rc('text', usetex=True)
font = {'family' : 'serif',
        'size'   : 16}
plt.rc('font', **font)
fig.set_size_inches(4,4)
ax.set_title(r'embedding RC')
ax.set_xlabel(r'$x_1$')
ax.set_ylabel(r'$x_2$')

plt.show()

In [ ]:
#THIS CELL CONTAINS SLICING AND INDEX SHAPESHIFTING EXAMPLES
#reshaping to N blocks of M endpoints corresponding to single start points
# row wise

a = np.arange(36)
A = a.reshape(6,6)
print(A)
# n_points - number of distributed startpoints
# M number of endpoints for each start point (= number of simulations per start point)
ix = np.arange(A.shape[0])
n_points = 3
M = int(A.shape[0]/n_points)
idx_new = ix.reshape(n_points, M).T.ravel()
print(A[idx_new, :].reshape(n_points, M, A.shape[1]))